Setup covid dataset

In [1]:
import os
os.chdir('../')
import pandas as pd

In [2]:
df=pd.read_csv('data/covid_moonshot/activity_data.csv')
df.head()

,SMILES,CID,canonical_CID,r_inhibition_at_20_uM,r_inhibition_at_50_uM,r_avg_IC50,f_inhibition_at_20_uM,f_inhibition_at_50_uM,f_avg_IC50,f_avg_pIC50,relative_solubility_at_20_uM,relative_solubility_at_100_uM,trypsin_IC50,NMR_std_ratio,acrylamide,chloroacetamide,series,frag_id
0,CCNC(=O)CN1CC2(CCN(c3cncc4ccccc34)C2=O)c2cc(Cl...,LUO-POS-e1dab717-11,LUO-POS-e1dab717-11,NaN,NaN,NaN,NaN,NaN,0.275118,NaN,NaN,NaN,NaN,NaN,False,False,3-aminopyridine-like,NaN
1,O=C(CN1CC2(CCN(c3cncc4ccccc34)C2=O)c2cc(Cl)ccc...,LUO-POS-e1dab717-12,LUO-POS-e1dab717-12,NaN,NaN,NaN,NaN,NaN,0.202767,NaN,NaN,NaN,NaN,NaN,False,False,3-aminopyridine-like,NaN
2,CNC(=O)C1(N2C[C@]3(CCN(c4cncc5ccccc45)C3=O)c3c...,MAT-POS-e48723dc-1,MAT-POS-e48723dc-1,NaN,NaN,NaN,NaN,NaN,55.453947,NaN,NaN,NaN,NaN,NaN,False,False,Ugi,NaN
3,CNC(=O)C1(N2C[C@@]3(CCN(c4cncc5ccccc45)C3=O)c3...,MAT-POS-e48723dc-2,MAT-POS-e48723dc-2,NaN,NaN,NaN,NaN,NaN,0.050000,NaN,NaN,NaN,NaN,NaN,False,False,Ugi,NaN
4,CNC(=O)CN1C[C@@]2(CCN(c3cncc4ccccc34)C2=O)c2cc...,LUO-POS-9931618f-2,LUO-POS-9931618f-2,NaN,NaN,NaN,NaN,NaN,0.052757,NaN,NaN,NaN,NaN,NaN,False,False,Ugi,NaN


In [3]:
df_actives_smiles=df[['SMILES','CID']][df['f_avg_IC50']<=5]
df_actives_smiles.head()
print(len(df_actives_smiles)/len(df)*100)

37.92434529582929


In [ ]:
df_actives_smiles.to_csv('data/covid_moonshot/actives_smiles.ism',index=False,sep=' ')
df_decoys_smiles=df[['SMILES','CID']][df['f_avg_IC50']>5]
df_decoys_smiles.to_csv('data/covid_moonshot/decoys_smiles.ism',index=False,sep=' ')

In [ ]:
import prody
import os

In [ ]:
pdb_dir='/home/rishal/pharmnn_rl/data/Mpro_XChem_screen_active_site_11-May-2020'
for file in os.listdir(pdb_dir):
    pdb=prody.parsePDB(pdb_dir+'/'+file)
    #remove water
    pdb=pdb.select('not water')
    #write out ligand to different pdb file
    ligand=pdb.select('resname LIG')
    prody.writePDB(pdb_dir+'/'+file.split('.pdb')[0]+'_lig.pdb',ligand)
    pdb=pdb.select('not resname LIG')
    prody.writePDB(pdb_dir+'/'+file.split('.pdb')[0]+'_nowat.pdb',pdb)

In [ ]:
from rdkit.Chem import AllChem
from rdkit import Chem

def fix_bond_order(template, pdb_ligand):
    #load molecule from template smiles
    template_mol=Chem.MolFromSmiles(template)
    #load molecule from pdb ligand
    pdb_ligand_mol=Chem.MolFromPDBFile(pdb_ligand)
    fixed_mol=AllChem.AssignBondOrdersFromTemplate(template_mol,pdb_ligand_mol)
    with Chem.SDWriter(pdb_ligand.split('.pdb')[0]+'_fixed.sdf') as w:
        w.write(fixed_mol)

In [ ]:
dataset_df=pd.read_excel('/home/rishal/pharmnn_rl/data/Mpro full XChem screen - hits summary - ver-2020-06-12.xlsx')
dataset_df.head()

In [ ]:
for file in os.listdir(pdb_dir):
    if file.endswith('lig.pdb'):
        mpro_id=file.split('_')[0]
        smiles=dataset_df['Compound SMILES'][dataset_df['Dataset']==mpro_id].values[0]
        try:
            fix_bond_order(smiles,pdb_dir+'/'+file)
        except:
            print(file)

In [ ]:
import openbabel as ob
from openbabel import pybel

In [ ]:

#infer aromaticity from molecules in sdf file
def infer_aromaticity(sdf_file):
    mol=pybel.readfile('sdf',sdf_file)
    for m in mol:
        m.write('sdf',sdf_file.split('.sdf')[0]+'_arom.sdf',overwrite=True)

In [ ]:
for file in os.listdir(pdb_dir):
    if file.endswith('fixed.sdf'):
        infer_aromaticity(pdb_dir+'/'+file)

Extract pharmacophores from all the ligand files

In [ ]:
pdb_dir='/home/rishal/pharmnn_rl/data/Mpro_XChem_screen_active_site_11-May-2020'
for file in os.listdir(pdb_dir):
    if file.endswith('arom.sdf'):
        os.system(pdb_dir+'/pharmit.2023 pharma -in '+pdb_dir+'/'+file+' -out '+pdb_dir+'/'+file.split('.sdf')[0]+'.json')

In [ ]:
import json
import pandas as pd

In [ ]:
def extract_json(json_file):
    """Takes a json files as input and returns a pandas dataframe with the headers as the keys of the json file"""
    with open(json_file) as f:
        data = json.load(f)
    data=data['points']
    df = pd.DataFrame(data)
    return df

In [ ]:
df_big=None
for file in os.listdir(pdb_dir):
    if file.endswith('.json'):
        df=extract_json(pdb_dir+'/'+file)
        if df_big is None:
            df_big=df
        else:
            df_big=df_big.append(df)

In [ ]:
print(df_big.head())

Cluster extracted pharmacophore points

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [ ]:
def cluster_xyz(feat_to_coords,distance_threshold=1.5):

    clustering = AgglomerativeClustering(n_clusters=None,compute_full_tree=True,linkage='average',distance_threshold=distance_threshold)
    for category in feat_to_coords.keys():
        if len(feat_to_coords[category])<2:
            continue
        final_xyz=[]
        coord_array=np.array(feat_to_coords[category])
        clustering.fit(coord_array)
        clusters=clustering.labels_
        for n in np.unique(clusters):
            cluster_center=coord_array[clusters==n].mean(axis=0)
            final_xyz.append(cluster_center)
        feat_to_coords[category]=final_xyz
    return feat_to_coords

In [ ]:
def write_xyz(feat_to_coords,xyz_file_name,pdb_dir):
    for feat in feat_to_coords:
        with open(pdb_dir+'/'+feat+'_'+xyz_file_name,'w') as f:
            for coord in feat_to_coords[feat]:
                f.write('H '+str(coord[0])+' '+str(coord[1])+' '+str(coord[2])+'\n')

In [ ]:
feat_to_coords={}
for feat,x,y,z in zip(df_big['name'],df_big['x'],df_big['y'],df_big['z']):
    if feat not in feat_to_coords.keys():
        feat_to_coords[feat]=[]
    coords=np.array([x,y,z])
    feat_to_coords[feat].append(coords)

In [ ]:
write_xyz(feat_to_coords,'original_unclustered.xyz',pdb_dir)

In [ ]:
print(feat_to_coords.keys())
print(len(feat_to_coords['Hydrophobic']))

In [ ]:
feat_to_coords=cluster_xyz(feat_to_coords,distance_threshold=1.5)
print(len(feat_to_coords['Aromatic']))

In [ ]:
write_xyz(feat_to_coords,'original_clustered.xyz',pdb_dir)

In [ ]:
print(len(feat_to_coords['Hydrophobic']))

In [ ]:
pdb_dir='/home/rishal/pharmnn_rl/data/Mpro_XChem_screen_active_site_11-May-2020'
lines=[]
file_write=open('data/covid_moonshot.txt','w')
for file in os.listdir(pdb_dir):
    if 'json' in file:
        if 'query' in file:
            continue
        df=extract_json(pdb_dir+'/'+file)
        for name,x,y,z in zip(df['name'],df['x'],df['y'],df['z']):
            file_write.write(name+','+str(x)+','+str(y)+','+str(z)+','+file.split('.json')[0] + '.sdf,' +file.split('_lig')[0]+'_nowat.pdb'+'\n')

In [6]:
example_ligand='Mpro-x0354_0_lig_fixed_arom.sdf'
example_protein='Mpro-x0354_0_nowat.pdb'

In [ ]:
def get_points_from_xyz(file):
    points=[]
    with open(file) as f:
        for line in f:
            points.append(line.split())
    return points

In [ ]:
file_write=open('data/covid_moonshot_clustered.txt','w')
for file in os.listdir(pdb_dir):
    if "orginal_clustered.xyz" in file:
        feature=file.split('_')[0]
        points=get_points_from_xyz(pdb_dir+'/'+file)
        for point in points:
            file_write.write(feature+','+point[1]+','+point[2]+','+point[3]+','+example_ligand+','+example_protein+'\n')

Put clustered points into a single json file

In [4]:
file_with_points='data/covid_moonshot_clustered.txt'
df=pd.read_csv(file_with_points,header=None)
df.columns=['feature','x','y','z','ligand','protein']
df.head()

,feature,x,y,z,ligand,protein
0,HydrogenDonor,6.1575,0.556,17.348500,Mpro-x0072_0_lig_fixed_arom.sdf,Mpro-x0072_0_nowat.pdb
1,HydrogenDonor,11.4465,5.834,23.068500,Mpro-x0072_0_lig_fixed_arom.sdf,Mpro-x0072_0_nowat.pdb
2,HydrogenDonor,3.6965,2.460,22.820000,Mpro-x0072_0_lig_fixed_arom.sdf,Mpro-x0072_0_nowat.pdb
3,HydrogenDonor,10.3530,1.571,24.379000,Mpro-x0072_0_lig_fixed_arom.sdf,Mpro-x0072_0_nowat.pdb
4,HydrogenDonor,7.5940,-0.516,21.114667,Mpro-x0072_0_lig_fixed_arom.sdf,Mpro-x0072_0_nowat.pdb


In [11]:
import json
json_file='data/covid_moonshot_clustered.json'
json_dict={}
json_dict['points']=[]
for row in df.iterrows():
    points_dict={}
    points_dict['name']=row[1]['feature']
    points_dict['x']=row[1]['x']
    points_dict['y']=row[1]['y']
    points_dict['z']=row[1]['z']
    points_dict['enabled']=True
    points_dict['radius']=1
    json_dict['points'].append(points_dict)

ligand=open('data/Mpro_XChem_screen_active_site_11-May-2020/'+ example_ligand).read()
protein=open('data/Mpro_XChem_screen_active_site_11-May-2020/'+example_protein).read()
json_dict['ligand']=ligand
json_dict['receptor']=protein
json_dict['recname']=example_protein
json_dict['ligname']=example_ligand
json_dict['ligandFormat']='sdf'
json.dump(json_dict,open(json_file,'w'))

Generate pharmacophore Queries

In [ ]:
import json, sys, os
from itertools import combinations

def genqueries(fname):
    prefix,ext = os.path.splitext(fname)
    q = json.load(open(fname))
    features = [feat for feat in q['points']]

    for i in range(3,len(features)+1):
        for j,combo in enumerate(combinations(features,i)):
            query = {"points": combo}
            json.dump(query, open(f'{prefix}_{i}_{j}.json','w'))

In [ ]:
pdb_dir='/home/rishal/pharmnn_rl/data/Mpro_XChem_screen_active_site_11-May-2020'

for file in os.listdir(pdb_dir):
    if 'json' in file:
        if 'query' in file:
            continue
        genqueries(pdb_dir+'/'+file)

##TODO Setup Fresco benchmark

In [ ]:
import pandas as pd
df_activity =pd.read_csv('data/fresco_moonshot/covid_moonshot_activity_data.csv')
df_activity.tail()

In [ ]:
len(df_activity)

In [ ]:
df_dates=pd.read_csv('data/fresco_moonshot/covid_moonshot_submission_dates.csv')
df_dates.head()
len(df_dates)

In [ ]:

df_dates=df_dates.sort_values(by='date')
df_dates.head()

In [ ]:
df_dates=df_dates[df_dates['date']<=20200901]
print(len(df_dates))

In [ ]:
df_dates=df_dates[df_dates['year']<=2020]
print(len(df_dates))

In [ ]:
df_dates=df_dates[df_dates['month']<=9]
print(len(df_dates))

In [ ]:
print(df_dates.head())
print(df_dates.tail())

In [ ]:
df_activity.head()
df_activity['canonical_CID']=df_activity['canonical_CID'].apply(lambda x: '-'.join(x.split('-')[:-1]))
df_activity.head()


In [ ]:
print(df_dates.head())

In [ ]:
df_activity_final=None
for submission in df_dates['submission_id']:
    df_temp=df_activity[df_activity['canonical_CID'].str.contains(submission)]
    if df_activity_final is None:
        df_activity_final=df_temp
    else:
        df_activity_final=df_activity_final.append(df_temp)

In [ ]:
df_activity_final.head()

In [ ]:
len(df_activity_final)

In [ ]:
print(len(df_activity_final[df_activity_final['f_avg_IC50']<=0.5]))

In [ ]:
print(df_activity_final[df_activity_final['canonical_CID']=='DAR-DIA-23aa0b97'])

In [ ]:
from rdkit.Chem import PandasTools

sdfFile = 'data/fresco_moonshot/covid_moonshot_docked_mols.sdf'
df_docking = PandasTools.LoadSDF(
    sdfFile, idName='canonical_CID', smilesName='SMILES', molColName='mol')

In [ ]:
from distutils.util import strtobool
df_docking = df_docking.dropna(subset=['acrylamide'])
df_docking['acrylamide'] = df_docking['acrylamide'].apply(
    strtobool).astype(bool)
df_docking['chloroacetamide'] = df_docking['chloroacetamide'].apply(
    strtobool).astype(bool)

df_docking = df_docking.query('~chloroacetamide & ~acrylamide').reset_index()
df_docking

In [ ]:

df_docking.f_avg_IC50 = df_docking.f_avg_IC50.astype(float)
df_docking['Chemgauss4 Score'] = df_docking['Chemgauss4 Score'].astype(float)

columns_to_keep = ['canonical_CID', 'Chemgauss4 Score', 'f_avg_IC50','SMILES']
df_docking = df_docking[columns_to_keep]
df_docking_grouped = df_docking.groupby(by=df_docking.canonical_CID)
df_docking_grouped = df_docking_grouped.agg({'SMILES':'first', 'Chemgauss4 Score':'mean', 'f_avg_IC50':'mean'})
df_docking_grouped['canonical_CID'] = df_docking_grouped.index

df_docking_grouped['hit'] = df_docking_grouped['f_avg_IC50'] < 5

In [ ]:
print(len(df_docking_grouped[df_docking_grouped['hit'] == True]))

In [ ]:
print(len(df_docking_grouped))

In [ ]:
print(df_docking_grouped.head())

In [ ]:
def remove_suffix(id):
    id_separated = id.split('-')[:-1]
    new_id = '-'.join(id_separated)
    return new_id
df_docking_grouped['submission_id'] = df_docking_grouped['canonical_CID'].apply(remove_suffix)
df_merged = df_docking_grouped.merge(df_dates, on='submission_id')
df_merged['Neg Dock Score'] = df_merged['Chemgauss4 Score']*-1

In [ ]:
print(len(df_docking_grouped))

In [ ]:
print(df_merged.head())
print(df_merged.tail())

In [ ]:
df_merged.sort_values(by='date', ascending=False).head()

In [ ]:
print(len(df_merged))

In [ ]:
year = '2020'
month = '09'
day = '01'
date_to_plot = int(f'{year}{month}{day}')
ic50_threshold = 5

#df_filtered_by_date = df_merged.query('date < @date_to_plot').copy()
df_filtered_by_date=df_merged[df_merged['date']<20200901]
df_filtered_by_date['hit'] = df_filtered_by_date['f_avg_IC50'] < ic50_threshold
print(len(df_filtered_by_date))

In [ ]:
print(len(df_filtered_by_date[df_filtered_by_date['hit'] == True]))

In [ ]:
df_actives=df_filtered_by_date[df_filtered_by_date['hit'] == True]
df_actives.head()

In [ ]:
df_actives=df_actives[['SMILES','canonical_CID']]
df_actives.head()

In [ ]:
df_actives.to_csv('data/fresco_moonshot/actives_smiles.ism',index=False,sep=' ')

In [ ]:
df_decoys=df_filtered_by_date[df_filtered_by_date['hit'] == False]
df_decoys=df_decoys[['SMILES','canonical_CID']]
df_decoys.to_csv('data/fresco_moonshot/decoys_smiles.ism',index=False,sep=' ')

Evaluate generated pharmacophores

In [ ]:
import subprocess
import pandas as pd
systems=[]
for file in os.listdir(pdb_dir):
    system=file.split('_')[0]
    if system not in systems:
        systems.append(system)
        scores={}
        for file in os.listdir(pdb_dir):
            if system in file:
                if 'json' in file:
                    output=subprocess.check_output('python getf1.py '+pdb_dir+'/'+file+' data/covid_moonshot/covid_moonshot/ --actives data/covid_moonshot/actives_smiles.ism --decoys data/covid_moonshot/decoys_smiles.ism',shell=True)
                    output=output.decode()
                    output=output.split(' ')
                    f1=output[1]
                    scores[file]=f1   
            else:
                continue
        df=pd.DataFrame.from_dict(scores,orient='index')
        df.to_csv(pdb_dir+'/'+system+'_scores.csv')
        print(system)
    else:
        continue

In [ ]:
import os
import pandas as pd
pdb_dir='/home/rishal/pharmnn_rl/data/Mpro_XChem_screen_active_site_11-May-2020'
for file in os.listdir(pdb_dir):
    if "scores.csv" in file:
        df=pd.read_csv(pdb_dir+'/'+file)
        try:
            df=df.sort_values(by='0',ascending=False)
        except:
            continue
        print(df.head())
        print(len(df[df['0']>0.46]),len(df))